In [6]:
import util
client = util.LLMClient(provider='gemini')

In [11]:
BINARY_LABEL = "Basophil"
# Initial meta prompt for the first iteration
other_labels = [label for label in util.CLASSES if label != BINARY_LABEL]
other_labels_str = ", ".join(other_labels)
MEDICAL_CONCEPTS_MAPPING = {
    "Basophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Blue/Black (dense)",
    "Eosinophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Red",
    "Lymphocyte": "Nucleus=Unsegmented, NC Ratio=High, Granularity=No, Size=Small",
    "Monocyte": "Nucleus=Unsegmented, NC Ratio=Low, Granularity=No",
    "Neutrophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Blue",
}


meta_init_prompt = f"""Give 50 distinct textual descriptions of pairs of visual discriminative features to identify whether the peripheral blood cell is a {BINARY_LABEL} or not. Other cell types include {other_labels_str}. 
These are the following features an expert would look for: Cell Size, Cell Shape, Nucleus Shape, Nuclear-Cytoplasmic Ratio, Chromatin-Density, Cytoplasm-Vacuole, Cytoplasm-Texture, Cytoplasm-Color, Granule-Type, Granule-Color, Granularity
{MEDICAL_CONCEPTS_MAPPING[BINARY_LABEL]}
The negative description should describe features common to all other cell types of {other_labels_str}.
The positive description should describe features specific to the {BINARY_LABEL}.
The negative, positive pairs should be discriminative.
Only provide the output as Python code in the following format: prompts = list[tuple[negative: str, positive: str]]. Let's think step-by-step"""

In [12]:
print(client.get_llm_response(meta_init_prompt))

```python
prompts = [
    ("Cell size is typically 10-12 μm.", "Cell size is often larger, exceeding 12 μm."),
    ("Cell shape is generally round or slightly irregular.", "Cell shape is frequently oval or kidney-bean shaped."),
    ("Nucleus is often round or oval with a smooth outline.", "Nucleus is frequently obscured by dense granules, appearing irregular or 'S' shaped."),
    ("Nuclear-cytoplasmic ratio is relatively high.", "Nuclear-cytoplasmic ratio is very low, with granules dominating the cell volume."),
    ("Chromatin is evenly distributed and moderately dense.", "Chromatin is often condensed and difficult to visualize due to granule coverage."),
    ("Cytoplasm lacks prominent vacuoles.", "Cytoplasm is filled with numerous, large, dark granules, obscuring any potential vacuoles."),
    ("Cytoplasm texture is generally homogenous or slightly granular.", "Cytoplasm texture is intensely granular, almost completely filled with granules."),
    ("Cytoplasm color is typically pal